<a href="https://colab.research.google.com/github/richardchen1210/licha/blob/master/stock_v4_%E6%8C%87%E5%AE%9A%E6%97%A5%E6%9C%9F%E7%9A%84%E5%89%8D280%E6%A0%B9(def%E7%89%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install mpl_finance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 480 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import yfinance as yf
import mpl_finance as mpf
import matplotlib as mpl
import matplotlib.pyplot as plt 
from matplotlib.font_manager import fontManager
import os
import csv

In [ ]:
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

# 改style要在改font之前
# plt.style.use('seaborn')  

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf') # 取用中文字型
mpl.rc('font', family='Taipei Sans TC Beta')

--2022-09-14 12:58:23--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 173.194.217.139, 173.194.217.113, 173.194.217.102, ...
Connecting to drive.google.com (drive.google.com)|173.194.217.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h9lai08sd4l2157nvo1fqo2sdd4225qc/1663160250000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_?uuid=05a662d6-5f98-4850-98d2-c8d367907138 [following]
--2022-09-14 12:58:27--  https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h9lai08sd4l2157nvo1fqo2sdd4225qc/1663160250000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_?uuid=05a662d6-5f98-4850-98d2-c8d367907138
Resolving doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to d

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gc

try:
    os.mkdir("/content/drive/MyDrive/stockhistory_kbar")
except:
    pass

# 根據data資料畫出一k線圖
def plot_280Kbar(data, stock_ind, stock_name):
    df = data

    fig = plt.figure(figsize=(24,8))  # 設定畫布大小(橫,直)
    ax = fig.add_subplot(1,1,1) # 等份切割畫布(直向,橫向,指定圖要放的位置) 假設切割成(2,3,1) 意指切成如2x3的陣列 位置編號[[1,2,3],[4,5,6]] 圖放在1號位置
    ax.set_xticks(range(0,len(df.index),30)) # 設定刻度 此為每30筆資料為一格
    #把日期後的時間刪掉(原始:2022-05-03 00:00:00) 只剩日期(2022-05-03) 由於資料量很多所有日期都顯示會擠在一起 所以只顯示每30天的日期
    convert_date = pd.DataFrame(df.index[::30])["Date"].apply(lambda x: x.strftime("%Y-%m-%d"))
    ax.set_xticklabels(convert_date, fontsize = 12) # x座標名稱
    # 繪製k線圖或稱蠟燭圖 主體(粗)端點代表開盤與收盤價 引線(細)的端點代表最高或最低
    # width 主體寬度  alpha 透明度  在台灣紅色(俗稱上漲 colorup="r")的由上到下:高收開低 綠色(下跌 color="g"):高開收低 (美國顏色相反)
    mpf.candlestick2_ochl(ax,df["Open"],df["Close"],df["High"],df["Low"],width = 0.6, colorup = "r", colordown = "g", alpha = 0.9)
    # 另有參數 rotation = 90 表刻度文字旋轉90度

    last_date = df.index[-1].strftime("%Y-%m-%d")
    print(last_date, end = " ")
    path = f"/content/drive/MyDrive/stockhistory_kbar/{stock_ind}"
    try:
        os.mkdir(path)
    except:
        pass
    plt.title(f"{stock_ind} {stock_name} {last_date}前280根k棒") # 標題名稱
    plt.xlabel("Date") # x軸標題
    plt.ylabel("Price") # y軸標題
    try:
        plt.savefig(f"/content/drive/MyDrive/stockhistory_kbar/{stock_ind}/{last_date}.png") # 要放在show前面 否則會變空圖
        plt.cla()
        plt.clf()
        plt.close()
        del fig
        del df
        del ax
        del path
        gc.collect(generation=1)
        gc.collect(generation=2)       
    except:
        pass
        
  


In [ ]:
# 連續繪製多圖使用 把資料切割成 n_bars 根k棒(預設280根) 再丟入plot_280Kbar畫圖

def cut_ndays_plot(data, stock_ind, stock_name, n_bars = 280):
    for i in range(len(data) - n_bars):
        if i == 0:
            df = data
        else:
            df = data[:-i]
        
        if len(df) < n_bars:
            break

        df = df[-n_bars:]
        plot_280Kbar(data = df, stock_ind = stock_ind, stock_name = stock_name)
        del df
        gc.collect(generation=1)
        gc.collect(generation=2) 

In [ ]:
with open("stocks_code.csv","r") as f:
    csvReader = csv.reader(f) # 讀取資料存於物件 此物件內容是list [[A,B],[C,D],...]
    dict_code = dict(csvReader) # 把csvReader物件轉成dict


In [ ]:
# 畫出所有股票歷史資料每日的前 n_bars 根k棒 應該是用不太到
for code in dict_code:

    print("目前股號:",code)

    data = yf.download(f"{code}.TW", period = "max", interval = "1d")
    cut_ndays_plot(data = data, stock_ind = code, stock_name = dict_code[code], n_bars = 280)

    del data
    gc.collect(generation=1)
    gc.collect(generation=2) 

    print("\n")


In [ ]:
# 畫出單日的前 280 根k棒 每日更新用
for code in dict_code:

    print("目前股號:",code)
    data = yf.download(f"{code}.TW", period = "max", interval = "1d")[-280:] # 取得最新資料的前280根
    plot_280Kbar(data = data, stock_ind = code, stock_name = dict_code[code])

    del data
    gc.collect(generation=1)
    gc.collect(generation=2) 

    print("\n")
